In [1]:
import os
import sys
import polars as pl
from typing import List, Dict, Callable
from dataclasses import dataclass
from main import DataDictionary
from variables import default_classifications

# 0. Description and classification of data objects

In [3]:
# Creates data dictionary
dictionary:DataDictionary = DataDictionary()
# Assigns default classifications
dictionary.apply_assign(default_classifications,dictionary.assign_classification)
# Prints the entries as a dataframe
dictionary_df:pl.DataFrame = dictionary.frame_dictionary()

# 1. Model the right target